# Preprocessamento de dados
### Ao final desta etapa teremos 4 conjunto de dados de acordo com o esquema abaixo:

<img src="https://user-images.githubusercontent.com/75253596/178109117-ed20f5d5-34c1-42da-a4f7-38d6ed688a07.png" width="400" height="600">

### imports

### loading data